In [27]:
import requests
from bs4 import BeautifulSoup
import csv

proxies = {
    'http': 'socks5h://127.0.0.1:9050',
    'https': 'socks5h://127.0.0.1:9050',
}

season = 2023
url = f"https://fbref.com/en/comps/12/{season}-{season+1}/schedule/{season}-{season+1}-La-Liga-Scores-and-Fixtures"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers, proxies=proxies)
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", {"class": "stats_table"})

with open(f"La_Liga_{season}_fixtures.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    headers = [header.text.strip() for header in table.find_all("th")]
    writer.writerow(headers)

    for row in table.find_all("tr"):
        cells = row.find_all("td")
        if cells:
            row_data = [cell.text.strip() for cell in cells]
            writer.writerow(row_data)

print(f"Table data has been written to 'La_Liga_{season}_fixtures.csv'")

Table data has been written to 'La_Liga_2023_fixtures.csv'


In [ ]:


url = f"https://fbref.com/en/comps/12/{season}-{season+1}/schedule/{season}-{season+1}-La-Liga-Scores-and-Fixtures"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers, proxies=proxies)
soup = BeautifulSoup(response.text, "html.parser")

soup


<!DOCTYPE html>

<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202502121" rel="dns-prefetch"/>
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp.osano.com/16CGnCU8UtNhM14sg/126698

In [24]:
df = all_matches

import pandas as pd

# Initialize Elo ratings
initial_elo = 1500
K = 30  # K-factor (controls rating volatility)

# Get all unique teams
teams = set(df["Home"]).union(set(df["Away"]))
elo_ratings = {team: initial_elo for team in teams}

# Function to compute expected probability
def expected_score(r_a, r_b):
    return 1 / (1 + 10 ** ((r_b - r_a) / 400))

# Function to update Elo ratings correctly
def update_elo(home, away, result):
    home_elo = elo_ratings[home]
    away_elo = elo_ratings[away]

    exp_home = expected_score(home_elo, away_elo)
    exp_away = expected_score(away_elo, home_elo)

    # Elo adjustment:
    if result == 0:  # Home team wins
        home_score, away_score = 1, 0
    elif result == 1:  # Away team wins
        home_score, away_score = 0, 1
    else:  # Draw
        home_score, away_score = 0.5, 0.5

    # Update ratings correctly
    elo_ratings[home] += K * (home_score - exp_home)
    elo_ratings[away] += K * (away_score - exp_away)

# Process each match in order
for _, row in df.iterrows():
    home_team = row["Home"]
    away_team = row["Away"]
    match_result = row["Result"]  # 0 = Home win, 1 = Away win, 0.5 = Draw

    update_elo(home_team, away_team, match_result)

# Convert Elo dictionary to sorted DataFrame
final_rankings = pd.DataFrame(elo_ratings.items(), columns=["Team", "Elo_Rating"])
final_rankings = final_rankings.sort_values(by="Elo_Rating", ascending=False).reset_index(drop=True)

# Display final rankings
print(final_rankings)


               Team   Elo_Rating
0         Barcelona  1813.950249
1       Real Madrid  1789.437844
2   Atlético Madrid  1699.079953
3           Sevilla  1664.291810
4          Valencia  1664.113259
5     Athletic Club  1598.542957
6        Villarreal  1560.422693
7        Celta Vigo  1548.556514
8     Real Sociedad  1529.925304
9          Espanyol  1504.754905
10                   1500.000000
11   Rayo Vallecano  1498.217179
12            Elche  1472.728064
13           Málaga  1471.084114
14          Levante  1452.266764
15        La Coruña  1451.897213
16          Granada  1451.349970
17          Osasuna  1447.134497
18         Mallorca  1444.793169
19   Sporting Gijón  1441.276005
20       Valladolid  1432.057104
21           Getafe  1414.946055
22            Eibar  1402.384406
23          Almería  1400.850933
24         Zaragoza  1399.538830
25         Hércules  1394.409974
26            Betis  1369.116963
27      Racing Sant  1357.677266
28          Córdoba  1325.196006
